In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import sklearn.linear_model as sk
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from collections import defaultdict
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from random import randint
from IPython.display import display
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn import preprocessing
import seaborn as sn
from sklearn.preprocessing import StandardScaler
from sklearn.semi_supervised import LabelPropagation
import warnings
warnings.filterwarnings('ignore')

In [2]:
#s = Data Path p = Number of Cluster col = Label Column Name
def detected_indices_temp(data,p,col,model):
    data['original'] = data[col]
    class0 = data.loc[data[col]!=1,:]
    class1 = data.loc[data[col]!=0,:]
    def noise(data):
        sample = data.sample(frac = 0.3)
        for i in list(sample.index):
            data[col][i] = int(not data[col][i])
        return list(sample.index)    
    noise0 = noise(class0)
    noise1 = noise(class1)
    noisy_indices = noise0+noise1
    mislabeled = pd.concat([class0,class1])
    indices = list(mislabeled.index)

    # User Optimal Number of Cluster
    km = KMeans(n_clusters = p)
    km.fit(mislabeled.loc[:,mislabeled.columns!=col])
    label = km.labels_
    mislabeled['cluster'] = label
    mislabeled['original'] = data['original']
    #OneHotEncoding
    mislabeled = pd.get_dummies(mislabeled,columns=['cluster'])

    
    #RandomForestClassifier
    x_train,x_test,y_train,y_test = train_test_split(mislabeled.loc[:,mislabeled.columns!=col],mislabeled[col],test_size = 0.38)
    rf = model()
    rf.fit(x_train,y_train)
    predict = rf.predict(mislabeled.loc[:,mislabeled.columns!=col])
    mislabeled['predicted'] = predict
    detected = list()
    cnt = 0
    
    ###########################################################################
#     print((mislabeled.loc[:,df.columns != [[col,'predicted']]],mislabeled[col]))
    
    for i in indices:
        if mislabeled[col][i] != mislabeled['predicted'][i]:
            mislabeled[col][i] = -1
    
    #Semisupervised
#     print(mislabeled[mislabeled.columns.difference([col,'predicted'])],mislabeled[col])
    lp = LabelPropagation()
    lp.fit(mislabeled[mislabeled.columns.difference([col,'predicted','original'])],mislabeled[col])
    arr = lp.predict(mislabeled[mislabeled.columns.difference([col,'predicted','original'])])
    
    mislabeled['semi'] = arr
    
    ############################################################
    for i in indices:
        if mislabeled[col][i] != mislabeled['semi'][i]:
            detected.append(i)
    bad_change =  0
    #detected Noise (Comparing with mislabled Data label)
    for i in detected:
        if i in noisy_indices:
            cnt+=1
        else:
            bad_change+=1
                
    return pd.DataFrame(data={'Total Mislabeled':[len(noisy_indices)], 
                              'Total Mismatch':[len(detected)], 
                              'True Identified labels(Actual Noise)':[cnt], 
                              'False identified labels(Not Noise fliped)':[bad_change], 
                              'Total Noise Ratio':len(noisy_indices)/data.shape[0], 
                              'True Identified Ratio(Actual Noise detected ratio)':cnt/len(noisy_indices)}
                       )

    
    #print(cnt,len(noisy_indices),cnt_noise)
    #print("Total Mislabeled labels", len(noisy_indices))
    #print("Mismatch with 'mislabled Data' label",cnt)
    #print("Mismatch with 'Original Data' label",cnt_noise)

In [3]:
data_df = pd.read_csv("../haberman.data",header=None)
data_df.fillna(data_df.median(),inplace=True)
data_df.columns = ['Age','Year','Nodes','Survival_status']
data_df['Survival_status'] = data_df['Survival_status'].astype(int)
data_df['Survival_status'] = data_df['Survival_status'].map({2:0 , 1:1})
data_df.drop_duplicates(inplace=True)

In [4]:
models = [SVC,RandomForestClassifier,GaussianNB,LogisticRegression,DecisionTreeClassifier]
#result = pd.DataFrame(data={'Total Noise':[], 'Total Detected labels':[], 'True Identified labels':[], 'False identified labels':[], 'Total Noise Ratio':[], 'True Identified Ratio':[]})
result = pd.DataFrame()
for model in models:
    temp = detected_indices_temp(data_df,14,'Survival_status',model)
    result = result.append(temp, ignore_index=True)
result['Model']=pd.Series(["SVC","Random Forest","Naive Bayes","Logistic Regression","Decision Tree"])
result

,Total Mislabeled,Total Mismatch,True Identified labels(Actual Noise),False identified labels(Not Noise fliped),Total Noise Ratio,True Identified Ratio(Actual Noise detected ratio),Model
0,87,239,63,176,0.301038,0.724138,SVC
1,87,196,41,155,0.301038,0.471264,Random Forest
2,87,229,66,163,0.301038,0.758621,Naive Bayes
3,87,231,84,147,0.301038,0.965517,Logistic Regression
4,87,51,22,29,0.301038,0.252874,Decision Tree
